# Statistical Modeling

### import libraries

In [1]:
import sys
sys.path.append('../src')
sys.path.append('../scripts')
from data_quality_check import missing_values_summary
import pandas as pd
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno

from scipy import stats
import statsmodels.api as sm
from encoding import encode_categorical_data,split_train_test
from models import train_models,evaluate_models

In [2]:
df=pd.read_csv('../data/cleaned_data.csv')

In [5]:
df['LossRatio'] = (df['TotalClaims'] / df['TotalPremium'])

In [9]:
df.head()

,UnderwrittenCoverID,PolicyID,TransactionMonth,IsVATRegistered,Citizenship,LegalType,Title,Language,Bank,AccountType,...,CoverCategory,CoverType,CoverGroup,Section,Product,StatutoryClass,StatutoryRiskType,TotalPremium,TotalClaims,LossRatio
0,145249,12827,2015-03-01,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Windscreen,Windscreen,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,21.929825,0.0,0.0
1,145249,12827,2015-05-01,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Windscreen,Windscreen,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,21.929825,0.0,0.0
2,145249,12827,2015-07-01,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Windscreen,Windscreen,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,0.000000,0.0,NaN
3,145255,12827,2015-05-01,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Own damage,Own Damage,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,512.848070,0.0,0.0
4,145255,12827,2015-07-01,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Own damage,Own Damage,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,0.000000,0.0,NaN


### Encoding categorical columns

In [3]:
categorical_columns =[
    'Citizenship', 'LegalType',
     'Gender',  'Province', 
    'MainCrestaZone', 'SubCrestaZone', 'ItemType', 'VehicleType', 
    'make', 'Model', 'bodytype', 'AlarmImmobiliser', 'TrackingDevice', 
    'TermFrequency', 'ExcessSelected', 'CoverCategory', 'CoverType', 
    'CoverGroup', 'Section', 'Product', 'StatutoryClass', 'StatutoryRiskType'
]
# Apply one-hot encoding
df_encoded = encode_categorical_data(df, categorical_columns, encoding_type='onehot')
df_encoded.head()

,UnderwrittenCoverID,PolicyID,TransactionMonth,IsVATRegistered,Title,Language,Bank,AccountType,MaritalStatus,Country,...,ohe_Standalone passenger liability,ohe_Third Party Only,ohe_Trailer,ohe_Motor Comprehensive,ohe_Optional Extended Covers,ohe_Standalone passenger liability,"ohe_Third party or third party, fire and theft only",ohe_Mobility Commercial Cover: Monthly,ohe_Mobility Metered Taxis: Monthly,ohe_Standalone Passenger Liability
0,145249,12827,2015-03-01,True,Mr,English,First National Bank,Current account,Not specified,South Africa,...,0,0,0,1,0,0,0,0,1,0
1,145249,12827,2015-05-01,True,Mr,English,First National Bank,Current account,Not specified,South Africa,...,0,0,0,1,0,0,0,0,1,0
2,145249,12827,2015-07-01,True,Mr,English,First National Bank,Current account,Not specified,South Africa,...,0,0,0,1,0,0,0,0,1,0
3,145255,12827,2015-05-01,True,Mr,English,First National Bank,Current account,Not specified,South Africa,...,0,0,0,1,0,0,0,0,1,0
4,145255,12827,2015-07-01,True,Mr,English,First National Bank,Current account,Not specified,South Africa,...,0,0,0,1,0,0,0,0,1,0


In [7]:
df_encoded.columns

Index(['UnderwrittenCoverID', 'PolicyID', 'TransactionMonth',
       'IsVATRegistered', 'Title', 'Language', 'Bank', 'AccountType',
       'MaritalStatus', 'Country',
       ...
       'ohe_Standalone passenger liability', 'ohe_Third Party Only',
       'ohe_Trailer', 'ohe_Motor Comprehensive',
       'ohe_Optional Extended Covers', 'ohe_Standalone passenger liability',
       'ohe_Third party or third party, fire and theft only',
       'ohe_Mobility Commercial Cover: Monthly',
       'ohe_Mobility Metered Taxis: Monthly',
       'ohe_Standalone Passenger Liability'],
      dtype='object', length=659)

#### Train-Test Split

In [5]:
# Define your features and target columns
features = ['ohe_Gauteng']  # Replace with your actual feature columns
target = 'TotalClaims'  # Replace with your actual target column

# Call the function to split your dataset
X_train, X_test, y_train, y_test = split_train_test(df_encoded, features, target, test_size=0.2, random_state=42)

# Display the first few rows of the training set
print('X-train \n',X_train.head())
print('Y-train \n',y_train.head())

X-train 
         ohe_Gauteng
352814            0
815361            1
129777            0
936713            1
283940            0
Y-train 
 352814    0.0
815361    0.0
129777    0.0
936713    0.0
283940    0.0
Name: TotalClaims, dtype: float64


In [6]:
# Assuming you have already split your data using the split_train_test function:
# X_train, X_test, y_train, y_test

# Train the models
models = train_models(X_train, y_train)

# Evaluate the models
evaluation_results = evaluate_models(models, X_test, y_test)

# Print the evaluation results
for model_name, metrics in evaluation_results.items():
    print(f"Model: {model_name}")
    print(f"Mean Absolute Error: {metrics['Mean Absolute Error']}")
    print(f"Mean Squared Error: {metrics['Mean Squared Error']}")
    print(f"R-Squared: {metrics['R-Squared']}")
    print("\n")


Model: Linear Regression
Mean Absolute Error: 126.93894841010444
Mean Squared Error: 5235619.803676664
R-Squared: 4.571430869737547e-06


Model: Decision Tree
Mean Absolute Error: 126.93894841010437
Mean Squared Error: 5235619.803676664
R-Squared: 4.5714308698485695e-06


Model: Random Forest
Mean Absolute Error: 126.67783314013649
Mean Squared Error: 5235615.451305678
R-Squared: 5.4027271159107215e-06


Model: XGBoost
Mean Absolute Error: 126.93894557150112
Mean Squared Error: 5235619.80362401
R-Squared: 4.571440926692816e-06


